In [ ]:
from model.model import build_model
from data.process import *
import keras

In [ ]:
tarin_df, dev_df, test_df = retrieve_train_dev_test_dataframe()


In [ ]:

X_train, y_train = split_input_output(tarin_df)

In [ ]:
model = build_model(len(X_train.columns), 64, 1)
opt = keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, batch_size=64, verbose=1)